In [2]:
# NE PAS TOUCHER!!

from IPython.display import Markdown
import ipywidgets as widgets
from ipywidgets import *
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import csv
import cufflinks as cf
import itertools
import scipy
from IPython.display import clear_output
# %matplotlib tk

# Read csv
os.chdir("C:/Users/paulc/Desktop/Stage/lmt-analysis-2022/LMT/scripts/Dataframes/")
df = pd.read_csv("Merge.csv")

date = df["Date"].unique()
injection = df["Injection"].unique()
cage = df["Cage"].unique()
night_phase = df["Night-Phase"].unique()
bins = df["Bin"].unique()
event = df["name"].unique()

reset_button = widgets.Button(description='Reset')
update_button = widgets.Button(description='Update')
# drop1 = widgets.SelectMultiple(options=date, rows=5, description='Date', disabled=False)
drop1 = widgets.SelectMultiple(options=injection, rows=5, description='Injection', disabled=False)
# drop1 = widgets.Dropdown(options=date, value=221013, description='Date:', disabled=False)
drop2 = widgets.SelectMultiple(options=cage, rows=5, description='Cage', disabled=False)
# drop2 = widgets.Dropdown(options=cage, value='Cage1', description='Cage:', disabled=False)
# drop3 = widgets.SelectMultiple(options=event, value=['Move'], rows=5, description='Event', disabled=False)
drop3 = widgets.Dropdown(options=event, value=event[1], description='Event:', disabled=False)
# drop4 = widgets.SelectMultiple(options=night_phase, rows=5, description='Night_Phase', disabled=False, value=[night_phase[0]])
drop4 = widgets.SelectMultiple(options=night_phase, rows=5, description='Night_Phase', disabled=False)
# drop4 = widgets.Dropdown(options=night_phase, value=1, description='Night_Phase:', disabled=False)

update_button.on_click(lambda b: update_options(None))

box = HBox([reset_button, update_button, drop1, drop2, drop4, drop3])

# On crée l'onglet pour les widgets
tab = widgets.Tab(children=[box])
tab.set_title(0, 'Widgets')

# On affiche l'onglet
display(tab)

# Test1

# def on_change_cage(change):
#     options = df['Date'][df['Cage'] == drop2.value].unique()
#     drop1.options = options
#     drop1.value = list(set(drop1.value) & set(options))

# drop2.observe(on_change_cage)

# def on_change_date(change):
#     options = df['Cage'][df['Date'].isin(drop1.value)].unique()
#     drop2.options = options
#     drop2.value = list(set(drop2.value) & set(options))

# drop1.observe(on_change_date)

# Test2

# first_call = True

# def on_change_date(change):
#     global first_call
#     drop2.options = df['Cage'][df['Date'] == drop1.value].unique()
#     drop4.options = df['Night-Phase'][df['Date'] == drop1.value].unique()
#     first_call = True

# drop1.observe(on_change_date)

# def on_change_cage(change):
#     global first_call
#     if first_call:
#         drop1.options = df['Date'][df['Cage'] == drop2.value].unique()
#         first_call = False
#     else:
#         drop1.value = ()
#         drop1.options = df['Date'][df['Cage'] == drop2.value].unique()

# drop2.observe(on_change_cage)

# def on_change(change):
#     drop2.options = df['Cage'][df['Date'] == drop1.value].unique()
#     drop4.options = df['Night-Phase'][df['Date'] == drop1.value].unique()
#     # drop1.options = df['Date'][df['Cage'] == drop2.value].unique()

# drop1.observe(on_change)
# # drop2.observe(on_change)

def reset_selections(b):
    drop1.value = ()
    drop2.options = df['Cage'].unique()
    drop4.options = df['Night-Phase'].unique()

reset_button.on_click(reset_selections)

# Pour sélectionner plusieurs Dates
def update_options(change):
    # Filtrer les données du DataFrame en fonction des valeurs sélectionnées dans drop1
    filtered_df = df[df['Injection'].isin(drop1.value)]
    
    # Mettre à jour les options de drop2 avec les valeurs uniques de la colonne "Cage" pour les données filtrées
    options = filtered_df['Cage'].unique()
    drop2.options = options
    
    # Conserver les valeurs sélectionnées dans drop2 si elles sont encore valides après la mise à jour des options
    drop2.value = list(set(drop2.value) & set(options))
    
    # Mettre à jour les options de drop4 avec les valeurs uniques de la colonne "Night-Phase" pour les données filtrées
    options = filtered_df['Night-Phase'].unique()
    drop4.options = options
    
    # Conserver les valeurs sélectionnées dans drop4 si elles sont encore valides après la mise à jour des options
    drop4.value = list(set(drop4.value) & set(options))

# Assigner la fonction update_options à l'événement "observe" de drop1
drop1.observe(update_options, names='value')

def update_temp_df(change):
    # Mise à jour de temp_df en fonction de la sélection actuelle des widgets
    global temp_df
    temp_df = df[(df["Injection"].isin((drop1.value))) & (df["Cage"].isin((drop2.value))) & (df["name"] == drop3.value) & (df["Night-Phase"].isin((drop4.value)))]

# Initialisation de temp_df en fonction de la sélection initiale des widgets
temp_df = df[(df["Injection"].isin((drop1.value))) & (df["Cage"].isin((drop2.value))) & (df["name"] == drop3.value) & (df["Night-Phase"].isin((drop4.value)))]

# Observation des widgets pour mettre à jour temp_df lorsque la sélection est modifiée
drop1.observe(update_temp_df, 'value')
drop2.observe(update_temp_df, 'value')
drop3.observe(update_temp_df, 'value')
drop4.observe(update_temp_df, 'value')

# Préparation pour le tracé
bins = temp_df['Bin'].unique()
animals = temp_df['RFidA'].unique()
nbrAnimals = len(animals)
nbrEvents = {}

# def update_dropdown(Date, Cage, Night_Phase, Event):
#     clear_output()
#     # info = f"La date choisie est le {date_val}, il s'agit de la {cage_val.lower()} pour l'évènement '{drop3.value}' durant la phase {night_phase_val} de nuit !"
#     display(Markdown(f"<h3>La date choisie est le {Date[0]}, il s'agit de la {Cage[0]} pour l'évènement '{Event}' durant la phase {Night_Phase[0]} de nuit !</h3>"))
#     global temp_df_loop
#     temp_df_loop = df[(df["Date"].isin(Date)) & (df["Cage"].isin(Cage)) & (df["name"].isin([Event])) & (df["Night-Phase"].isin(Night_Phase))]
#     # plt.figure(figsize=(18, 6))
#     # fig, (line_ax, bar_ax) = plt.subplots(1, 2, figsize=(20,5))
#     # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), width_ratios=[1, 1])
#     fig, ax1 = plt.subplots(figsize=(12,6))
#     ax2 = fig.add_axes([1, 0.25, 0.25, 0.5])
#     # x=temp_df_loop[temp_df_loop.RFidA.isin(temp_df_loop.RFidA.unique())]['Bin']
#     # y=temp_df_loop[temp_df_loop.RFidA.isin(temp_df_loop.RFidA.unique())]["numberOfEvents"]
#     sns.lineplot(temp_df_loop, y='numberOfEvents', x='Bin', hue="GenoA", style='RFidA', palette={'Amphet':'red', 'NaCl':'blue'}, ax=ax1)
#     ax1.set_title(f'Numbers of {drop3.value} per bin of each mouse')
#     ax1.set_ylabel(f'Numbers of {drop3.value}')
#     sns.barplot(data=temp_df_loop, y='numberOfEvents', x="GenoA", palette={'Amphet':'red', 'NaCl':'blue'}, ax=ax2)
#     ax2.set_title("A barplot")
#     plt.show()
#     mean_AMPHET = temp_df_loop[temp_df_loop["GenoA"] == 'Amphet']["numberOfEvents"]
#     mean_NACL = temp_df_loop[temp_df_loop["GenoA"] == 'NaCl']["numberOfEvents"]
#     stat, pvalue = scipy.stats.ttest_ind(mean_NACL,mean_AMPHET)
#     display(Markdown(f"<h3>La statistique du t-test est de {stat} et la p-value est de {pvalue}</h3>"))

# Fonction pour afficher le plot en fonction des choix de l'utilisateur
def update_dropdown(date, cage, night_phase, event):
    clear_output()
    for date_val, cage_val, night_phase_val in itertools.product(date, cage, night_phase):
        # info = f"La date choisie est le {date_val}, il s'agit de la {cage_val.lower()} pour l'évènement '{drop3.value}' durant la phase {night_phase_val} de nuit !"
        display(Markdown(f"<h3>La date choisie est le {date_val}, il s'agit de la {cage_val} pour l'évènement '{drop3.value}' durant la phase {night_phase_val} de nuit !</h3>"))
        global temp_df_loop
        temp_df_loop = df[(df["Injection"] == date_val) & (df["Cage"] == cage_val) & (df["name"] == drop3.value) & (df["Night-Phase"] == night_phase_val)]
        # plt.figure(figsize=(18, 6))
        # fig, (line_ax, bar_ax) = plt.subplots(1, 2, figsize=(20,5))
        # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), width_ratios=[1, 1])
        fig, ax1 = plt.subplots(figsize=(12,6))
        ax2 = fig.add_axes([1, 0.25, 0.25, 0.5])
        # x=temp_df_loop[temp_df_loop.RFidA.isin(temp_df_loop.RFidA.unique())]['Bin']
        # y=temp_df_loop[temp_df_loop.RFidA.isin(temp_df_loop.RFidA.unique())]["numberOfEvents"]
        sns.lineplot(temp_df_loop, y='numberOfEvents', x='Bin', hue="GenoA", style='RFidA', palette={'Amphet':'red', 'NaCl':'blue'}, ax=ax1)
        ax1.set_title(f'Numbers of {drop3.value} per bin of each mouse')
        ax1.set_ylabel(f'Numbers of {drop3.value}')
        sns.barplot(data=temp_df_loop, y='numberOfEvents', x="GenoA", palette={'Amphet':'red', 'NaCl':'blue'}, ax=ax2)
        ax2.set_title("A barplot")
        plt.show()
        mean_AMPHET = temp_df_loop[temp_df_loop["GenoA"] == 'Amphet']["numberOfEvents"]
        mean_NACL = temp_df_loop[temp_df_loop["GenoA"] == 'NaCl']["numberOfEvents"]
        stat, pvalue = scipy.stats.ttest_ind(mean_NACL,mean_AMPHET)
        display(Markdown(f"<h3>La statistique du t-test est de {stat} et la p-value est de {pvalue}</h3>"))

# # #interact function in isolation
# adapt_plot = interact(update_dropdown, date=drop1, cage=drop2, event=drop3, night_phase=drop4)
# # tab_nest.children = [VBox(children = adapt_plot.children)]
# # display(tab_nest)
# adapt_plot

interactive_plot = widgets.interactive_output(update_dropdown, {'date': drop1, 'cage': drop2, 'night_phase': drop4, 'event': drop3})
display(interactive_plot)

# Faire un on_change de Date quand on clique sur la Cage après avoir choisi la Date

C:\Users\paulc\AppData\Local\Temp\ipykernel_4500\4282371521.py:21: DtypeWarning:

Columns (18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.



Output()